<a href="https://colab.research.google.com/drive/renato/Criando-seu-Ecossistema-de-Big-Data-na-Nuvem/blob/main/Exemplo_Coleta_Dados_Spark_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALAÇÃO DAS BIBLIOTECAS AZURE-STORAGE-BLOB E PYSPARK

In [ ]:
pip install azure-storage-blob

     |████████████████████████████████| 356 kB 3.2 MB/s 
     |████████████████████████████████| 179 kB 39.8 MB/s 
     |████████████████████████████████| 3.6 MB 49.4 MB/s 
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 41 kB 568 kB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 46.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bbd504ecd19519822f1299217dc52c874b604e82cec1b5419d13dac11b3062d5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import requests
import azure.storage.blob

In [ ]:
def download(url_arquivo, nome_arquivo):
  req = requests.get(url_arquivo)
  conteudo = req.content
  arquivo = open(nome_arquivo, 'wb')
  arquivo.write(conteudo)
  arquivo.close()

In [ ]:
url_ordem_compra = 'https://datalakeaula1377299.blob.core.windows.net/datalake/raw-zone/olist/olist_orders_dataset/olist_orders_dataset.csv'
url_cliente ='https://datalakeaula1377299.blob.core.windows.net/datalake/raw-zone/olist/olist_customers_dataset/olist_customers_dataset.csv'
nome_arquivo_ordem_compra = 'olist_orders_dataset.csv'
nome_arquivo_cliente = 'olist_customers_dataset.csv'
download(url_ordem_compra, nome_arquivo_ordem_compra)
download(url_cliente, nome_arquivo_cliente)

In [ ]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
schema_orders = StructType([ \
    StructField('order_id', StringType(), True), \
    StructField('customer_id', StringType(), True), \
    StructField('order_status', StringType(), True), \
    StructField('order_purchase_timestamp', TimestampType(), True), \
    StructField('order_approved_at', TimestampType(), True), \
    StructField('order_delivered_carrier_date', TimestampType(), True), \
    StructField('order_delivered_customer_date', TimestampType(), True), \
    StructField('order_estimated_delivery_date', TimestampType(), True) \
])

In [ ]:
df_orders = spark.read.csv(nome_arquivo_ordem_compra, header=True, inferSchema=True, schema=schema_orders)

In [ ]:
df_clientes = spark.read.csv(nome_arquivo_cliente, header=True, inferSchema=True)

In [ ]:
df_clientes.show(truncate=False, n=5)

+--------------------------------+--------------------------------+------------------------+---------------------+--------------+---------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city        |customer_state|customer_region|
+--------------------------------+--------------------------------+------------------------+---------------------+--------------+---------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|861eff4711a542e4b93843c6dd7febb0|14409                   |franca               |SP            |Sudeste        |
|18955e83d337fd6b2def6b18a428ac77|290c77bc529b7ac935b93aa66c333dc3|9790                    |sao bernardo do campo|SP            |Sudeste        |
|4e7b3e00288586ebd08712fdd0374a03|060e732b5b29e8181a18229c7b0b2b5e|1151                    |sao paulo            |SP            |Sudeste        |
|b2b6027bc5c5109e529d4dc6358b12c3|259dac757896d24d7702b9acbbff3f3c|8775                    |mogi das cruzes      |SP        

In [ ]:
df_orders.show(truncate=False, n=3)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [ ]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [ ]:
# df_orders.agg({"order_purchase_timestamp": "max"}).show()

In [ ]:
df_orders.createOrReplaceTempView("df_orders")

In [ ]:
df_clientes.createOrReplaceTempView("df_clientes")

In [ ]:
df_clientes = spark.sql("""
SELECT  *,
        CASE
          WHEN customer_state IN ('MG', 'SP', 'RJ', 'ES') THEN 'Sudeste'
          WHEN customer_state IN ('RS', 'SC', 'PR') THEN 'Sul'
          WHEN customer_state IN ('DF', 'GO', 'MS', 'MT') THEN 'Centro-Oeste'
          WHEN customer_state IN ('AM', 'AP', 'RR', 'RO', 'PA', 'AC', 'TO') THEN 'Norte'
          ELSE 'Nordeste'
        END AS customer_region
FROM    df_clientes
""")

In [ ]:
df_clientes.createOrReplaceTempView("df_clientes_2")

In [ ]:
spark.sql("""
SELECT    customer_region,
          customer_state,
          COUNT(*) AS quantidade_clientes
FROM      df_clientes_2
GROUP BY  ROLLUP(customer_region,
          customer_state)
ORDER BY  customer_region,
          quantidade_clientes desc
""").show(truncate=False, n=1000)

+---------------+--------------+-------------------+
|customer_region|customer_state|quantidade_clientes|
+---------------+--------------+-------------------+
|null           |null          |99441              |
|Centro-Oeste   |null          |5782               |
|Centro-Oeste   |DF            |2140               |
|Centro-Oeste   |GO            |2020               |
|Centro-Oeste   |MT            |907                |
|Centro-Oeste   |MS            |715                |
|Nordeste       |null          |9394               |
|Nordeste       |BA            |3380               |
|Nordeste       |PE            |1652               |
|Nordeste       |CE            |1336               |
|Nordeste       |MA            |747                |
|Nordeste       |PB            |536                |
|Nordeste       |PI            |495                |
|Nordeste       |RN            |485                |
|Nordeste       |AL            |413                |
|Nordeste       |SE            |350           